## Анализ набора данных с использованием `pandas` (2)

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('Employee.csv')

In [3]:
df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [4]:
df.groupby('EverBenched')['LeaveOrNot'].mean()*100

EverBenched
No     33.125749
Yes    45.397490
Name: LeaveOrNot, dtype: float64

Постройте сводную таблицу, показывающую средний возраст сотрудников в каждом городе.

In [5]:
age_by_city = df.pivot_table(values='Age', index='City', aggfunc='mean')
print(age_by_city)

                 Age
City                
Bangalore  29.594255
New Delhi  29.121003
Pune       29.288644


In [6]:
df.pivot_table(values=['Age'], index=['City'])

,Age
City,
Bangalore,29.594255
New Delhi,29.121003
Pune,29.288644


In [7]:
df.groupby('City')['Age'].mean()

City
Bangalore    29.594255
New Delhi    29.121003
Pune         29.288644
Name: Age, dtype: float64

Рассчитайте количество сотрудников по каждому уровню образования и их средний возраст.

In [8]:
df.groupby('Education')['Age'].agg('mean')

Education
Bachelors    29.422938
Masters      29.290951
PHD          29.296089
Name: Age, dtype: float64

In [9]:
df3 = df.groupby('Education')['City'].count()
print('Количество сотрудников: \n', df3.head())
print()
df4 = df.groupby('Education')['Age'].mean()
print('Средний возраст:  \n', df4.head())

Количество сотрудников: 
 Education
Bachelors    3601
Masters       873
PHD           179
Name: City, dtype: int64

Средний возраст:  
 Education
Bachelors    29.422938
Masters      29.290951
PHD          29.296089
Name: Age, dtype: float64


In [10]:
grouped_df = df.groupby('Education')['Age']
result = grouped_df.agg(['size', 'mean']).reset_index()
result.columns = ['Education', 'NumberOfEmployees', 'AverageAge']
print(result)

   Education  NumberOfEmployees  AverageAge
0  Bachelors               3601   29.422938
1    Masters                873   29.290951
2        PHD                179   29.296089


In [11]:
import numpy.ma as ma

df.pivot_table(values=['Age'], index=['Education'], aggfunc=[ma.count, np.mean])

/var/folders/24/z61d72zx27z5ml8twybhwmm40000gn/T/ipykernel_63914/27923053.py:3: FutureWarning: The provided callable <function mean at 0x10d77fec0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.pivot_table(values=['Age'], index=['Education'], aggfunc=[ma.count, np.mean])


,count,mean
,Age,Age
Education,,
Bachelors,3601,29.422938
Masters,873,29.290951
PHD,179,29.296089


In [12]:
df.groupby('Education').agg({
    'Education': 'count',
    'Age': 'mean'})

,Education,Age
Education,,
Bachelors,3601,29.422938
Masters,873,29.290951
PHD,179,29.296089


In [14]:
df.groupby('Education').agg(num_emp=('Education', 'count'), avg_age=('Age', 'mean'))

,num_emp,avg_age
Education,,
Bachelors,3601,29.422938
Masters,873,29.290951
PHD,179,29.296089


Создайте сводную таблицу, показывающую количество мужчин и женщин в каждом уровне оплаты (Payment Tier).

In [15]:
df.pivot_table(values=['Age'], index=['PaymentTier', 'Gender'], aggfunc=[ma.count])

count
                     Age
PaymentTier Gender      
1           Female   131
            Male     112
2           Female   605
            Male     313
3           Female  1139
            Male    2353

In [ ]:
pivot_table = pd.crosstab(index=df['PaymentTier'], columns=df['Gender'])
print(pivot_table)

KeyError: ('Gender', 'City')

In [17]:
df.pivot_table(values = 'PaymentTier', index = 'Gender', aggfunc = 'count')

,PaymentTier
Gender,
Female,1875
Male,2778


In [42]:
pivot=pd.pivot_table(df,index='PaymentTier', columns='City', values='Age',aggfunc='count')
pivot

City,Bangalore,New Delhi,Pune
PaymentTier,,,
1,112,56,75
2,65,353,500
3,2051,748,693


Составьте сводную таблицу, отражающую процент сотрудников, покинувших компанию в следующие два года, в зависимости от того, были ли они когда-либо исключены из проектов на месяц или более.

In [44]:
df.groupby('EverBenched')['LeaveOrNot'].value_counts(normalize=True)

EverBenched  LeaveOrNot
No           0             0.668743
             1             0.331257
Yes          0             0.546025
             1             0.453975
Name: proportion, dtype: float64

In [45]:
result = df.groupby('EverBenched')['LeaveOrNot'].value_counts(normalize=True).unstack()*100
result.columns = ['Оставшиеся (%)','Покинувшие (%)']
result = result[['Покинувшие (%)']]
result.index.name = None
result.columns.name = None
result

,Покинувшие (%)
No,33.125749
Yes,45.397490


Постройте сводную таблицу, отображающую медианный опыт работы в текущей области в зависимости от уровня образования и пола сотрудников.

In [23]:
pivot=pd.pivot_table(df,index='Education',columns='Gender',values='ExperienceInCurrentDomain',aggfunc='median')
pivot

Gender,Female,Male
Education,,
Bachelors,3.0,3.0
Masters,2.0,3.0
PHD,3.0,3.0


In [24]:
df.pivot_table(
    values='ExperienceInCurrentDomain',  
    index=['Education', 'Gender'],       
    aggfunc='median'                     
)

ExperienceInCurrentDomain
Education Gender                           
Bachelors Female                        3.0
          Male                          3.0
Masters   Female                        2.0
          Male                          3.0
PHD       Female                        3.0
          Male                          3.0

In [25]:
df.pivot_table(values=['ExperienceInCurrentDomain'], index=['Education', 'Gender'], aggfunc=[np.median])

/var/folders/24/z61d72zx27z5ml8twybhwmm40000gn/T/ipykernel_63914/429795385.py:1: FutureWarning: The provided callable <function median at 0x10d8a6660> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df.pivot_table(values=['ExperienceInCurrentDomain'], index=['Education', 'Gender'], aggfunc=[np.median])


median
                 ExperienceInCurrentDomain
Education Gender                          
Bachelors Female                       3.0
          Male                         3.0
Masters   Female                       2.0
          Male                         3.0
PHD       Female                       3.0
          Male                         3.0

Рассчитайте количество сотрудников по каждому году присоединения к компании и их средний возраст.

In [26]:
empl=df.groupby('JoiningYear').agg(count_empl=('Age','count'),mean_empl=('Age','mean'))
empl

,count_empl,mean_empl
JoiningYear,,
2012,504,29.603175
2013,669,29.008969
2014,699,29.290415
2015,781,29.476312
2016,525,29.552381
2017,1108,29.416968
2018,367,29.525886


In [48]:
result = df.groupby('JoiningYear').agg(
    Count=('Age', 'count'),  
    AverageAge=('Age', 'mean')  
).reset_index()
result

,JoiningYear,Count,AverageAge
0,2012,504,29.603175
1,2013,669,29.008969
2,2014,699,29.290415
3,2015,781,29.476312
4,2016,525,29.552381
5,2017,1108,29.416968
6,2018,367,29.525886


Создайте сводную таблицу, показывающую процент сотрудников, покинувших компанию в следующие два года, в зависимости от уровня образования и города, в котором они работают.

In [49]:
df[df['LeaveOrNot'] == 1].pivot_table(values = "LeaveOrNot", index = ["Education", "City"], aggfunc = lambda x: sum(x) / sum(df['LeaveOrNot']) * 100)

LeaveOrNot
Education City                 
Bachelors Bangalore     30.8125
          New Delhi      6.0625
          Pune          33.6875
Masters   Bangalore      5.4375
          New Delhi     15.1875
          Pune           6.0000
PHD       Bangalore      0.9375
          New Delhi      1.6250
          Pune           0.2500

In [51]:
df.groupby(['Education', 'City'])['LeaveOrNot'].value_counts(normalize=True)

Education  City       LeaveOrNot
Bachelors  Bangalore  0             0.759747
                      1             0.240253
           New Delhi  0             0.819367
                      1             0.180633
           Pune       1             0.532609
                      0             0.467391
Masters    Bangalore  1             0.701613
                      0             0.298387
           New Delhi  0             0.529981
                      1             0.470019
           Pune       0             0.586207
                      1             0.413793
PHD        Bangalore  0             0.711538
                      1             0.288462
           New Delhi  0             0.747573
                      1             0.252427
           Pune       0             0.833333
                      1             0.166667
Name: proportion, dtype: float64

In [52]:
result = df.groupby(['Education', 'City'])['LeaveOrNot'].value_counts(normalize=True).unstack(fill_value=0)*100
result.columns = ['Осталось (%): \n','Покинуло (%)']
result = result[['Покинуло (%)']]
result.index.name = None
result.columns.name = None
result

Покинуло (%)
Education City                   
Bachelors Bangalore     24.025341
          New Delhi     18.063315
          Pune          53.260870
Masters   Bangalore     70.161290
          New Delhi     47.001934
          Pune          41.379310
PHD       Bangalore     28.846154
          New Delhi     25.242718
          Pune          16.666667

Составьте сводную таблицу, отображающую средний возраст сотрудников, которые были когда-либо исключены из проектов, и тех, кто не был, в зависимости от уровня оплаты.

In [35]:
pivot_table = df.pivot_table(
    values='Age', 
    index='PaymentTier', 
    columns='EverBenched',  
    aggfunc='mean' 
).reset_index()
pivot_table.columns.name = None  
pivot_table = pivot_table.rename(columns={
    'No': 'Средний возраст (не исключались из проектов)',
    'Yes': 'Средний возраст (исключались из проектов)'
})
pivot_table

,PaymentTier,Средний возраст (не исключались из проектов),Средний возраст (исключались из проектов)
0,1,29.644444,28.944444
1,2,29.204848,28.989247
2,3,29.460160,29.217984


In [53]:
pivot=pd.pivot_table(df,index='PaymentTier',columns='EverBenched',values='Age',aggfunc='mean').round(1)
pivot

EverBenched,No,Yes
PaymentTier,,
1,29.6,28.9
2,29.2,29.0
3,29.5,29.2


In [55]:
print('Количество сотрудников \n', df.pivot_table(
    values='Age',  
    index=['EverBenched', 'PaymentTier'],       
    aggfunc='mean'                     
))

Количество сотрудников 
                                Age
EverBenched PaymentTier           
No          1            29.644444
            2            29.204848
            3            29.460160
Yes         1            28.944444
            2            28.989247
            3            29.217984


In [56]:
df.pivot_table(values = "Age", index = "LeaveOrNot", columns = "PaymentTier", aggfunc = ['mean'])

mean                      
PaymentTier          1          2          3
LeaveOrNot                                  
0            29.974026  29.755435  29.520743
1            28.932584  28.800000  29.208117

In [57]:
df.pivot_table(values = "Age", index = "EverBenched", columns = "PaymentTier", aggfunc = ['mean'])

mean                      
PaymentTier          1          2          3
EverBenched                                 
No           29.644444  29.204848  29.460160
Yes          28.944444  28.989247  29.217984

In [59]:
df.pivot_table(values=['Age'], index=['PaymentTier', 'EverBenched'], aggfunc='mean')

Age
PaymentTier EverBenched           
1           No           29.644444
            Yes          28.944444
2           No           29.204848
            Yes          28.989247
3           No           29.460160
            Yes          29.217984

Постройте сводную таблицу, показывающую средний возраст сотрудников, которые покинули компанию в следующие два года, в зависимости от города и опыта работы в текущей области.

In [64]:
filtered_df = df.query('LeaveOrNot==1')
pivot_table = filtered_df.pivot_table(
    values='Age',
    index='City',            
    columns='ExperienceInCurrentDomain',  
    aggfunc='mean',
    fill_value=0             
)
pivot_table

ExperienceInCurrentDomain,0,1,2,3,4,5,6,7
City,,,,,,,,
Bangalore,30.822222,30.975309,29.413793,28.669492,28.115789,28.905660,38.5,39.0
New Delhi,30.000000,30.090909,29.724771,27.552239,27.472973,27.984375,0.0,0.0
Pune,31.833333,31.283784,28.406897,28.359649,27.898438,28.728814,0.0,0.0


In [65]:
df[df['LeaveOrNot'] == 1].pivot_table(values = "Age", index = ["City"], columns = ['ExperienceInCurrentDomain'], aggfunc = ['mean'])

mean                                   \
ExperienceInCurrentDomain          0          1          2          3   
City                                                                    
Bangalore                  30.822222  30.975309  29.413793  28.669492   
New Delhi                  30.000000  30.090909  29.724771  27.552239   
Pune                       31.833333  31.283784  28.406897  28.359649   

                                                             
ExperienceInCurrentDomain          4          5     6     7  
City                                                         
Bangalore                  28.115789  28.905660  38.5  39.0  
New Delhi                  27.472973  27.984375   NaN   NaN  
Pune                       27.898438  28.728814   NaN   NaN

In [66]:
print('Средний возраст сотрудников \n', df.pivot_table(
    values='Age',  
    index=['City', 'ExperienceInCurrentDomain', 'LeaveOrNot'],       
    aggfunc='mean'                     
))
df6 = df.pivot_table(
    values='Age',  
    index=['City', 'ExperienceInCurrentDomain', 'LeaveOrNot'],       
    aggfunc='mean'                     
)
print("=======================================================")
print ('Размер: \n')
print (df6.shape)

Средний возраст сотрудников 
                                                       Age
City      ExperienceInCurrentDomain LeaveOrNot           
Bangalore 0                         0           32.212121
                                    1           30.822222
          1                         0           31.085308
                                    1           30.975309
          2                         0           28.863192
                                    1           29.413793
          3                         0           28.992674
                                    1           28.669492
          4                         0           29.014045
                                    1           28.115789
          5                         0           29.674352
                                    1           28.905660
          6                         0           34.500000
                                    1           38.500000
          7                         0     

In [68]:
df_leave=df[df['LeaveOrNot']==1]
pivot=pd.pivot_table(df_leave,index='City',columns='ExperienceInCurrentDomain',values='Age',aggfunc='mean').round(1)
pivot

ExperienceInCurrentDomain,0,1,2,3,4,5,6,7
City,,,,,,,,
Bangalore,30.8,31.0,29.4,28.7,28.1,28.9,38.5,39.0
New Delhi,30.0,30.1,29.7,27.6,27.5,28.0,NaN,NaN
Pune,31.8,31.3,28.4,28.4,27.9,28.7,NaN,NaN


Создайте сводную таблицу, отображающую процент сотрудников, покинувших компанию в следующие два года, в зависимости от уровня оплаты и того, были ли они когда-либо исключены из проектов на месяц или более.

In [69]:
df.groupby(['EverBenched', 'PaymentTier'])[['EverBenched', 'PaymentTier', 'LeaveOrNot']].value_counts(normalize=True)

EverBenched  PaymentTier  LeaveOrNot
No           1            0             0.644444
                          1             0.355556
             2            1             0.575758
                          0             0.424242
             3            0             0.735040
                          1             0.264960
Yes          1            0             0.500000
                          1             0.500000
             2            1             0.806452
                          0             0.193548
             3            0             0.637602
                          1             0.362398
Name: proportion, dtype: float64

In [70]:
df[df['LeaveOrNot'] == 1].pivot_table(values = "LeaveOrNot", 
                                      index = ["PaymentTier"], 
                                      columns = ["EverBenched"], 
                                      aggfunc = lambda x: sum(x) / sum(df['LeaveOrNot']) * 100)

EverBenched,No,Yes
PaymentTier,,
1,5.0000,0.5625
2,29.6875,4.6875
3,51.7500,8.3125


In [71]:
pivot=pd.pivot_table(df,index='PaymentTier',columns='EverBenched',values='LeaveOrNot',aggfunc='mean').multiply(100).round(1)
pivot

EverBenched,No,Yes
PaymentTier,,
1,35.6,50.0
2,57.6,80.6
3,26.5,36.2
